In [1]:
import numpy as np
import pandas as pd

In [2]:
class SPP:
    def __init__(self, grammar, nTerminals, terminals):
        self.grammar = grammar
        self.nTerminals = nTerminals
        self.terminals = terminals 
        self.first = dict()
        self.last = dict() 
        
        
        size = len(nTerminals) + len(terminals) 
        self.mat = np.zeros((size,size), dtype=str)
        self.update_matrix()
        
        self.name = dict()
        self.update_name()
        
        self.grammar_list = list()
        # Amguity check or just adds the inversed grammar to the list
        self.ambiguity_check_n_inversed_grammar()
        
        
    def update_matrix(self):
        b = np.array(['<'] *(len(self.terminals) + len(self.nTerminals)))
        self.mat = np.vstack((self.mat ,b))
        b = np.array(['>'] *(  len(b) + 1)) 
        self.mat = np.hstack((self.mat,b.reshape(len(b),1)))  
        
    def show_matrix(self):
        df  = pd.concat( [pd.DataFrame( (self.nTerminals + self.terminals + ['$']) ),  pd.DataFrame(self.mat, columns = (self.nTerminals + self.terminals + ['$'] ))], axis = 1)
        print(df) 
        

    def ambiguity_check_n_inversed_grammar(self):

        inversed_grammar = list()
        for prod in self.grammar:
            if len(self.grammar[prod]) == 1:
                inversed_grammar.append([self.grammar[prod][0],prod] )
                continue
            for x in self.grammar[prod]:
                inversed_grammar.append([x,prod]) 


        counter = dict()
        for x in inversed_grammar:
            if x[0] in counter.keys():
                counter[x[0]] += 1
                continue
            counter[x[0]] = 1 


        index_dict = dict()
        for i, prod in enumerate(inversed_grammar):
            if counter[prod[0]] > 1 and prod[0] in index_dict.keys():
                index_dict[prod[0]].append(i)
                continue
            if counter[prod[0]] > 1:
                index_dict[prod[0]] =  [i] 


        parse_len = len(index_dict)  
        node_list = list()
        
        if parse_len > 1:
            k = 0 
            while k < parse_len:
                if k == 0:
                    for x in index_dict[list(index_dict.keys())[ parse_len - 2 ]]:
                        for y in index_dict[list(index_dict.keys())[ parse_len - 1 ]]:
                            node_list.append([x,y])

                    k += 2
                    continue

                new_node_list = list()

                for  x in index_dict[list(index_dict.keys())[ parse_len - 1 - k]]:
                    for y in node_list:
                        new_node_list.append([x] + y) 
                node_list = new_node_list

                del new_node_list
                k += 1

        if parse_len == 1:
            for x in index_dict[ list(index_dict.keys())[0]]:
                node_list.append([x])  

        if parse_len == 0:
            node_list.append([]) 


        for index_arr in node_list:
            k = 0 
            new_grammar = inversed_grammar.copy()
            for index, index_elem in enumerate(index_arr):
                for y in index_dict [ list( index_dict.keys())[index]]:
                    if y == index_elem:
                        continue
                    new_grammar.pop(y - k)
                    k += 1
            self.grammar_list.append(new_grammar)

        
    def first_last_assign(self,nTerminal, last = False): 
        non_terminals_queue = [nTerminal] 
        temp = list() 

        while bool(non_terminals_queue):
            if non_terminals_queue[0].islower() and non_terminals_queue[0] in temp:
                non_terminals_queue.pop(0) 
                continue  
            if non_terminals_queue[0].islower() and non_terminals_queue[0] not in temp:
                temp.append(non_terminals_queue[0])
                non_terminals_queue.pop(0) 
                continue 

            productions = grammar[non_terminals_queue[0]]
            for prod in productions:
                char = prod[0]
                if last:
                    char = prod[-1]
                if char not in temp:
                    non_terminals_queue.append(char)
                    temp.append(char)

            non_terminals_queue.pop(0) 

        return temp
    
    def first_last_init(self):
        for nTerminal in self.nTerminals:   
            self.first[nTerminal], self.last[nTerminal] = self.first_last_assign(nTerminal), self.first_last_assign(nTerminal,True)
    
    def update_name(self):
        for x,i in zip ( (self.nTerminals + self.terminals), range(len(self.nTerminals+ self.terminals))):
            self.name[x] = i 
        self.name['$'] = len(self.name)
            
    
    def apply_rules(self):
        for key in self.grammar:
                for prod in self.grammar[key]:
                    if len(prod) > 1:
                        for index, char in enumerate(prod):
                            self.eq_rule(index,char,prod)
                            self.smaller_rule(index,char,prod)
                            self.gr_1(index,char,prod)
                            self.gr_2(index,char,prod) 
        
    def eq_rule(self,index,char,prod):
        if index + 1 < len(prod):
    
            self.mat [ self.name[char], self.name[ prod[index + 1] ]] = '=' 
            
    def smaller_rule(self,index,char,prod):
        if (index + 1) < len(prod) and prod[index + 1].isupper():
            for x in self.first[prod[index + 1]]:
                 self.mat[ self.name[char], self.name[x] ] = '<' 
                    
    
    def gr_1(self,index,char,prod):
        if char.isupper() and index + 1 < len(prod)  and prod[index + 1].islower():
            for x in self.last[char]:
                self.mat[ self.name[x], self.name[prod[index+1]] ] = '>' 
                    
                    
    def gr_2(self,index,char,prod):
        if  char.isupper() and index + 1 < len(prod) and prod[index+1].isupper():
            for x in self.last[char]:
                counter = 0
                for y in self.first[prod[index+1]]:
                    if y.islower():
                        counter +=1 
                        a[ name[x], name[y] ] = '>'
                if counter == 0:
                    print('##### No terminals found on First(LEFT) !!')
                    break 
                    
                    
    def init_string(self, string):
        new_string ="$"
        for i, char in enumerate(string[1:],1):
            new_string += self.mat[ self.name[string[i - 1]], self.name[char] ]
            new_string += char
        return new_string 
    
    
    def get_indexes(self,string):
        low = None
        high = None
        for i,char  in enumerate(string):
            if char == '<':
                low = i+1
            if char == '>' and low != None:
                high = i
                break
        if bool(low) and bool(high):
            return low,high
        return None,None 
    
    
    
    def update_string(self,string,grammar):
        n = dict()
        for x in grammar:
            n[x[0]]  = x[1]

        x,y = self.get_indexes(string)
        if x == None and y == None:
            return None

        new_string = ''
        word = string[x:y]
        if len(word) > 1:
            word =''
            for z in range(x,y,2):
                word += string[z]

        prod_replaced = None
        try:
            prod_replaced =  n[word]
        except:
            print("Precidence symbol in table does not exist.  Derivation is above. Grammar:")
            print(grammar)
            print('\n\n')
            return None


        left =  string[:x-1] 
        left_relation = self.mat[ self.name[string[x-2:x-1]], self.name[ prod_replaced ] ]
        right_relation = self.mat[  self.name [ prod_replaced ],  self.name[string[y+1:y+2]]] 
        right = string[y+1:]
        new_string = left + left_relation + prod_replaced + right_relation + right
        print(new_string)
        if new_string == '$<S>$':
            print(grammar)
            print('############################################## SUCCESS ABOVE #####################################') 

        return new_string 
    
    
    def main(self):
        self.first_last_init()
        self.apply_rules()
        
        for grammar in self.grammar_list:
            string = "$abacdae$"
            string = self.init_string(string)
            print(string)

            while string != '$<S>$':
                string = self.update_string(string,grammar)
                if string == None:
                    break



In [3]:
grammar = {
    "S": ["C"],
    "C" : ["BcA"],
    "A" : ["b", "dD"],
    "D": ["Be", 'a'], # added extra a for ambiguity check
    "B": ["a", "Bba"]
}

nTerminals = ["S","A","B", "C", "D"]
terminals = ["a","b", "c", "d", "e"]

In [4]:
a = SPP(grammar, nTerminals, terminals)

In [5]:
a.main()

$<a>b=a>c<d<a>e>$
$<Db=a>c<d<a>e>$
Precidence symbol in table does not exist.  Derivation is above. Grammar:
[['C', 'S'], ['BcA', 'C'], ['b', 'A'], ['dD', 'A'], ['Be', 'D'], ['a', 'D'], ['Bba', 'B']]



$<a>b=a>c<d<a>e>$
$<B=b=a>c<d<a>e>$
$<B=c<d<a>e>$
$<B=c<d<B=e>$
$<B=c<d=D>$
$<B=c=A>$
$<C>$
$<S>$
[['C', 'S'], ['BcA', 'C'], ['b', 'A'], ['dD', 'A'], ['Be', 'D'], ['a', 'B'], ['Bba', 'B']]
############################################## SUCCESS ABOVE #####################################


In [6]:
a.show_matrix()

    0  S  A  B  C  D  a  b  c  d  e  $
0   S                                >
1   A                                >
2   B                    =  =     =  >
3   C                                >
4   D                                >
5   a                    >  >     >  >
6   b                 =              >
7   c     =              <     <     >
8   d        <     =  <              >
9   e                                >
10  $  <  <  <  <  <  <  <  <  <  <  >
